In [ ]:
# Como estamos trabajando en colab y tengo los archivos en el drive importamos la libreria y montamos
from google.colab import drive #para utilizar google drive como fuente de archivos
drive.mount('/content/drive') #montamos la carpeta del drivec

Mounted at /content/drive


In [ ]:
!pip install -U googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40711 sha256=3c451fbfc0da85e0eda18f8d07a92cc64bc5c7469ca441a3dac191d4296e1f0b
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [ ]:
!pip install pyarrow

In [ ]:
import googlemaps
import polars as pl

In [ ]:
# Leer el archivo Parquet desde GCS
df = pl.read_parquet('/content/drive/MyDrive/PF Google & Yelp/1.parquet')

In [ ]:
# Configurar la API Key de Google Maps
with open('/content/drive/MyDrive/PF Google & Yelp/APK.txt', 'r') as file:
    api_key = file.read().strip()

gmaps = googlemaps.Client(key=api_key)

In [ ]:
df.columns

['name',
 'gmap_id',
 'latitude',
 'longitude',
 'category',
 'avg_rating',
 'num_of_reviews',
 'url']

In [ ]:
df.dtypes

[Utf8, Utf8, Float64, Float64, Utf8, Float64, Int64, Utf8]

In [ ]:
df[0,2:4]

latitude,longitude
f64,f64
41.219586,-111.954159


In [ ]:
resultado = gmaps.reverse_geocode((41.219586, -111.954159))

In [ ]:
resultado[0]['place_id']

'ChIJzXI05CoPU4cR-ycyvxzibgc'

In [ ]:
# Definir la función para obtener city y country a partir de coordenadas
def obtener_city_country(latitudes, longitudes):
    cities, countries, place_ids = [], [], []

    for lat, lon in zip(latitudes, longitudes):
        resultado = gmaps.reverse_geocode((lat, lon))
        city, country, place_id = None, None, None

        if resultado:
            for component in resultado[0]['address_components']:
                if 'administrative_area_level_1' in component['types'] and not city:
                    city = component['long_name']
                elif 'country' in component['types'] and not country:
                    country = component['long_name']

            if resultado[0]['place_id']:
                place_id = resultado[0]['place_id']

        cities.append(city)
        countries.append(country)
        place_ids.append(place_id)

    return pl.Series(cities), pl.Series(countries), pl.Series(place_ids)

In [ ]:
# Aplicar la función a las columnas 'latitude' y 'longitude'
cities, countries, place_ids = obtener_city_country(df['latitude'], df['longitude'])

In [ ]:
# Agregar las nuevas series al DataFrame
df = df.with_columns(
    City=cities,
    Country=countries,
    Place_ID=place_ids
)

In [ ]:
df

name,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,url,City,Country,Place_ID
str,str,f64,f64,str,f64,i64,str,str,str,str
"""browning mansi…","""0x87530f2ae430…",41.219586,-111.954159,"""Hotel""",5.0,1,"""https://www.go…","""Utah""","""United States""","""ChIJzXI05CoPU4…"
"""Knights Inn Fr…","""0x8094677ef8ae…",36.791747,-119.792588,"""Hotel""",2.2,66,"""https://www.go…","""California""","""United States""","""ChIJ84Jp_X5nlI…"
"""Golden Nugget""","""0x863b85806de0…",30.251152,-93.179729,"""Hotel""",4.0,1,"""https://www.go…","""Louisiana""","""United States""","""ChIJTwPy7XiEO4…"
"""Bearcove Resor…","""0x52b711dd0516…",46.94733,-94.535447,"""Resort hotel""",3.3,4,"""https://www.go…","""Minnesota""","""United States""","""ChIJc5lgBd0Rt1…"
"""Hollywood""","""0x89e60e6029e2…",41.353617,-72.094278,"""Hotel""",4.0,1,"""https://www.go…","""Connecticut""","""United States""","""ChIJsZvKK2AO5o…"
"""Lazy K Lounge""","""0x89c89ce228f7…",40.030569,-76.504901,"""Hotel""",4.0,1,"""https://www.go…","""Pennsylvania""","""United States""","""ChIJm5D3KOKcyI…"
"""Posey Knob""","""0x88862b383e7b…",33.697157,-87.595702,"""Resort hotel""",5.0,1,"""https://www.go…","""Alabama""","""United States""","""GhIJt5bJcDzZQE…"
"""Campbell Colle…","""0x862833975d49…",32.298545,-90.209289,"""Hotel""",4.5,2,"""https://www.go…","""Mississippi""","""United States""","""ChIJcwbCaZczKI…"
"""Boathouse Barn…","""0x8669e732cdb6…",28.413809,-96.719294,"""Hotel""",4.4,8,"""https://www.go…","""Texas""","""United States""","""ChIJ6RX70TLnaY…"


In [ ]:
df.write_parquet('/content/drive/MyDrive/PF Google & Yelp/u1.parquet')